In [1]:
import akshare as ak 

import pandas as pd
import numpy as np

import os 
from datetime import datetime
from tqdm import tqdm

In [19]:
# load the hongli list
hs = pd.read_csv('../data/input/honglidibo_list_20251213.csv')
hs['code'] = hs['code'].astype(str).str.zfill(6)

STOCK_CODES = hs['code'].tolist()
print(len(STOCK_CODES))

50


In [20]:
# check financial data already loaded
STOCK_LOADED = os.listdir('../data/input/financial-indicators/20251212/')
STOCK_LOADED = [stock[21:27] for stock in STOCK_LOADED]
 
len(STOCK_LOADED)

2886

In [21]:
len([stock for stock in STOCK_CODES if stock not in STOCK_LOADED])

23

In [22]:
# check financial data already loaded
STOCK_LOADED = os.listdir('../data/input/financial-indicators/20251213/')
STOCK_LOADED = [stock[21:27] for stock in STOCK_LOADED]
 
len(STOCK_LOADED)

361

In [23]:
MISSED_CODE = [stock for stock in STOCK_CODES if stock not in STOCK_LOADED]
len(MISSED_CODE)

13

In [24]:
today = pd.to_datetime("today").strftime("%Y%m%d")

while len(MISSED_CODE) > 0:
    print(len(MISSED_CODE))
    for stock_code in tqdm(MISSED_CODE):
        try: 
            # --- get the financial data ---
            financial_df = ak.stock_financial_abstract_ths(symbol=f"{stock_code}", indicator="按单季度")
            # select the key indicators
            financial_df = financial_df[['报告期', '每股净资产', '基本每股收益', '净资产收益率']]
            # rename the columns
            financial_df.columns = ['report_date', 'bps', 'eps', 'roe']
            # chage the date format
            financial_df['report_date'] = pd.to_datetime(financial_df['report_date'])
            # choose the date later than 2010-01-01
            financial_df = financial_df[financial_df['report_date'] >= '2010-01-01']
            # change the data format
            financial_df['eps'] = financial_df['eps'].astype(float)
            financial_df['roe'] = financial_df['roe'].str.replace('%', '').astype(float)
            financial_df['bps'] = financial_df['bps'].astype(float)
            # calculate ttm eps and ttm roe
            financial_df['bps_ttm'] = financial_df['bps'].rolling(window=4).mean()
            financial_df['eps_ttm'] = financial_df['eps'].rolling(window=4).sum()
            financial_df['roe_ttm'] = financial_df['roe'].rolling(window=4).sum()
            # drop the values with null values
            financial_df.dropna(inplace=True)
            
            # --- merge the financial data with standardized report dates ---
            # standardize the report dates
            date_df = pd.DataFrame(pd.date_range(start='2010-12-31', end='2025-12-31', freq='ME'), columns=['report_date'])
            financial_date = pd.merge(date_df, financial_df, on='report_date', how='left', validate="1:1")
            financial_date.to_csv(f"../data/input/financial-indicators/20251213/financial_indicators_{stock_code}_{today}.csv", index=False)
            MISSED_CODE.remove(stock_code)
        except: 
            # MISSED_CODE.append(stock_code)
            continue

13


 54%|█████▍    | 7/13 [00:17<00:15,  2.53s/it]


7


 57%|█████▋    | 4/7 [00:13<00:10,  3.48s/it]


4


...

 67%|██████▋   | 2/3 [00:03<00:01,  1.96s/it]


1


100%|██████████| 1/1 [00:07<00:00,  7.32s/it]


1


100%|██████████| 1/1 [00:01<00:00,  1.03s/it]
